In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/den/dev/git/ozon-e-cup-2025


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline

import src.preprocessing as prep

In [ ]:
train = pd.read_csv("data/1__train_with_flags.csv")
test = pd.read_csv("data/1__test_with_flags.csv")

Обучу все трансформы как так train, так и на test и запишу в csv.

In [ ]:
# Переведем SellerID в строку (надо для корректной работы Categorical трансформа)
train["SellerID"] = train["SellerID"].astype(str)
test["SellerID"] = test["SellerID"].astype(str)

In [ ]:
# Сформируем train + test для обучения
train_test = pd.concat([train, test], axis=0)

train.shape, test.shape, train_test.shape

((197198, 48), (22760, 47), (219958, 48))

In [ ]:
# Создаём пайплайн препроцессинга
pipeline = Pipeline(
    [("numeric", prep.NUMERIC_PREP_PIPELINE), ("categorical", prep.CATEGORIAL_PREP_PIPELINE)], verbose=True
)

In [ ]:
# Обучаем
pipeline = pipeline.fit(train_test)

[Pipeline] ... (step 1 of 37) Processing rating_1_count, total=   1.9s
[Pipeline] ... (step 2 of 37) Processing rating_2_count, total=   0.8s
[Pipeline] ... (step 3 of 37) Processing rating_3_count, total=   0.3s
[Pipeline] ... (step 4 of 37) Processing rating_4_count, total=   0.4s
[Pipeline] ... (step 5 of 37) Processing rating_5_count, total=   0.2s
[Pipeline]  (step 6 of 37) Processing comments_published_count, total=   0.2s
[Pipeline]  (step 7 of 37) Processing photos_published_count, total=   0.4s
[Pipeline]  (step 8 of 37) Processing videos_published_count, total=   0.2s
[Pipeline] .. (step 9 of 37) Processing PriceDiscounted, total=   0.2s
[Pipeline] . (step 10 of 37) Processing item_time_alive, total=   0.2s
[Pipeline]  (step 11 of 37) Processing item_count_fake_returns7, total=   0.2s
[Pipeline]  (step 12 of 37) Processing item_count_fake_returns30, total=   0.3s
[Pipeline]  (step 13 of 37) Processing item_count_fake_returns90, total=   0.2s
[Pipeline]  (step 14 of 37) Proces

In [ ]:
# Применяем
train_processed = pipeline.transform(train)
test_processed = pipeline.transform(test)

/home/den/miniconda3/envs/ozon-e-cup/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/den/miniconda3/envs/ozon-e-cup/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [ ]:
# Сохраняем
train_processed.to_csv("data/2__train_with_flags_num_cat.csv", index=False)
test_processed.to_csv("data/2__test_with_flags_num_cat.csv", index=False)

## Проверим

In [12]:
added_columns_train = train_processed.shape[1] - train.shape[1]
added_columns_test = test_processed.shape[1] - test.shape[1]
assert added_columns_train == added_columns_test
print(f"Columns added in processed: {added_columns_train}")

Columns added in processed: 31


In [ ]:
categorial_cols = [
    "brand_name",  # мб Nan: добавил флаг
    "CommercialTypeName4",  # всегда есть и не пусто как в train так и в test
    "SellerID",  # всегда есть и не пусто как в train так и в test
]
numeric_cols = [  # числовые фичи, NaN значения заменяем на -1 (перед Scaler)
    "rating_1_count",
    "rating_2_count",
    "rating_3_count",
    "rating_4_count",
    "rating_5_count",
    "comments_published_count",
    "photos_published_count",
    "videos_published_count",
    "PriceDiscounted",
    "item_time_alive",
    "item_count_fake_returns7",
    "item_count_fake_returns30",
    "item_count_fake_returns90",
    "item_count_sales7",
    "item_count_sales30",
    "item_count_sales90",
    "item_count_returns7",
    "item_count_returns30",
    "item_count_returns90",
    "GmvTotal7",
    "GmvTotal30",
    "GmvTotal90",
    "ExemplarAcceptedCountTotal7",
    "ExemplarAcceptedCountTotal30",
    "ExemplarAcceptedCountTotal90",
    "OrderAcceptedCountTotal7",
    "OrderAcceptedCountTotal30",
    "OrderAcceptedCountTotal90",
    "ExemplarReturnedCountTotal7",
    "ExemplarReturnedCountTotal30",
    "ExemplarReturnedCountTotal90",
    "ExemplarReturnedValueTotal7",
    "ExemplarReturnedValueTotal30",
    "ExemplarReturnedValueTotal90",
    "ItemVarietyCount",
    "ItemAvailableCount",
    "seller_time_alive",
]

In [ ]:
for cc in categorial_cols:
    print(f"Column: {cc}")
    for c in train_processed.columns:
        if c.startswith(cc):
            print(c)
    print()

Column: brand_name
brand_name
brand_name_0
brand_name_1
brand_name_2
brand_name_3
brand_name_4
brand_name_5
brand_name_6
brand_name_7
brand_name_8
brand_name_9

Column: CommercialTypeName4
CommercialTypeName4
CommercialTypeName4_0
CommercialTypeName4_1
CommercialTypeName4_2
CommercialTypeName4_3
CommercialTypeName4_4
CommercialTypeName4_5
CommercialTypeName4_6
CommercialTypeName4_7
CommercialTypeName4_8

Column: SellerID
SellerID
SellerID_0
SellerID_1
SellerID_2
SellerID_3
SellerID_4
SellerID_5
SellerID_6
SellerID_7
SellerID_8
SellerID_9
SellerID_10
SellerID_11



In [ ]:
for nc in numeric_cols:
    print(f"Column: {nc}")
    if nc in train_processed.columns:
        print(nc)
    else:
        raise Exception(f"Column {nc} not found in processed data!")
    print()

Column: rating_1_count
rating_1_count

Column: rating_2_count
rating_2_count

Column: rating_3_count
rating_3_count

Column: rating_4_count
rating_4_count

Column: rating_5_count
rating_5_count

Column: comments_published_count
comments_published_count

Column: photos_published_count
photos_published_count

Column: videos_published_count
videos_published_count

Column: PriceDiscounted
PriceDiscounted

Column: item_time_alive
item_time_alive

Column: item_count_fake_returns7
item_count_fake_returns7

Column: item_count_fake_returns30
item_count_fake_returns30

Column: item_count_fake_returns90
item_count_fake_returns90

Column: item_count_sales7
item_count_sales7

Column: item_count_sales30
item_count_sales30

Column: item_count_sales90
item_count_sales90

Column: item_count_returns7
item_count_returns7

Column: item_count_returns30
item_count_returns30

Column: item_count_returns90
item_count_returns90

Column: GmvTotal7
GmvTotal7

Column: GmvTotal30
GmvTotal30

Column: GmvTotal90
GmvT

In [ ]:
train_processed.head()

,resolution,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,...,SellerID_2,SellerID_3,SellerID_4,SellerID_5,SellerID_6,SellerID_7,SellerID_8,SellerID_9,SellerID_10,SellerID_11
0,0,ACTRUM,"Мешки пылесборники для пылесоса PHILIPS, 10 шт...","Мешки для пылесоса PHILIPS TRIATLON, синтетиче...",Пылесборник,1.759945,2.598419,1.850168,1.373086,1.559950,...,0,0,0,0,0,0,0,0,0,1
1,0,Red Line,Защитная силиконовая крышка обьектива GoPro He...,Защитная крышка Redline на экшн-камеру GoPro (...,Крышка для объектива,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,0,0,0,0,0,0,0,0,1,0
2,0,<RARE_BRAND_NAME>,Плоский медиатор из кости толщиной 0.6 мм<br/>...,Медиатор для гитары Acura GP-PB6,Аксессуар для музыкального инструмента,-0.654425,-0.444209,0.492633,-0.557058,-0.650023,...,0,0,0,0,0,0,0,0,1,1
3,0,NaN,"Игра Sonic Frontiers для PlayStation 5, русски...","Игра Sonic Frontiers для PlayStation 5, русски...",Видеоигра,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,0,0,0,0,0,0,0,1,0,0
4,0,NaN,Disney Classic Games: Aladdin and The Lion Kin...,"Игра Aladdin and Lion King (PlayStation 4, анг...",Видеоигра,0.205591,-0.444209,-0.534302,-0.557058,-1.196451,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
test_processed.head()

,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,...,SellerID_2,SellerID_3,SellerID_4,SellerID_5,SellerID_6,SellerID_7,SellerID_8,SellerID_9,SellerID_10,SellerID_11
0,NaN,Приветствую! Мы собираем классные системные бл...,"LeBefane Системный блок (AMD Ryzen 5 7500F, RA...",Настольный компьютер,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,0,1,0,0,0,0,0,1
1,Canon,<li>Тип товара Новый</li><li><b>Тип</b> Картри...,Картридж лазерный Canon 071H 5646C002 черный (...,"Картридж, чернила, тонер",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,1,1,1,0,0,0,0,1,1
2,Devia,Электронное перо Touch S-Pen для Samsung Galax...,Стилус для Samsung Galaxy Z Fold5 5G/ SM-F946B...,Стилус,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,0,0,0,1,0,1,1,0
3,GALAXY LINE,Мультиварка GALAXY GL2643 имеет все необходимы...,"Мультиварка GALAXY GL2643 ( 900 Вт, 10 програм...","Мультиварка, скороварка",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,1,1,1,1,1,0,1,0,0
4,MAUNFELD,Чайники с ретро-термометром позволяет нагреть ...,Чайник электрический MAUNFELD MGK-625MINT. Тов...,Чайник электрический,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,1,1,1,1
